如果要加入新資料
1. NewDataLoader(BasicLoader)
2. LoaderMapping新增欄位
3. .yml新增欄位

In [1]:
### reload magic
%reload_ext autoreload
%autoreload 2

import sys
import yaml
import torch
import torchvision
import torch.utils.cpp_extension

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, DeviceStatsMonitor

from src.data_manager import DataManager
from src.model_director import ModelDirector

# os.environ['CUDA_VISIBLE_DEVICES']="0"
print('Python version', sys.version)
print('CUDA_HOME', torch.utils.cpp_extension.CUDA_HOME)
print('CudaToolKit Version', torch.version.cuda)
print('Cudnn Version', torch.backends.cudnn.version())
print('torch Version', torch.__version__)
print('torchvision Version', torchvision.__version__)

config_file = './exp_config/exp2.yml'
with open(config_file, "r") as content:
    config = yaml.safe_load(content) 

Python version 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
CUDA_HOME /usr/local/cuda
CudaToolKit Version 11.8
Cudnn Version 8700
torch Version 2.0.0+cu118
torchvision Version 0.15.1+cu118


In [2]:
dm = DataManager(
    data_meta_info=config['train_config']['data_meta_info'], 
    **config['train_config']['data_loader_params']
)

100%|██████████| 362024/362024 [00:02<00:00, 157964.65it/s]


RainLoaderJay: time list is loaded.


100%|██████████| 366871/366871 [00:02<00:00, 150490.14it/s]


RadarLoaderJay: time list is loaded.
[DataManager] Training Data Size: 10192, Validating Data Size: 3396, Testing Data Size: 3396 
[DataManager] Image Shape: (540, 420), Batch Size: 8, Sampling Rate: 3


In [3]:
model = ModelDirector(
    data_info = dm.get_data_info(),
    **config['model'],
).build_model()

[EncoderParams] channel_count:2 Shape:(540, 420)
[ForecasterPONI] TargetLen:3 TeacherForcing:0.5
[Discriminator] Downsample:5
[ModelDirector] Using CPN_PONI model


In [4]:
logger = TensorBoardLogger(save_dir='logs', name=config['model']['model_config']['name'])
checkpoint_callback = model.get_checkpoint_callback()
trainer = Trainer(
    benchmark=True,
    accelerator='gpu',
    devices=[0],
    fast_dev_run=False, # True: debug and compile once 
    logger=logger,
    check_val_every_n_epoch=1,
    max_epochs=2, 
    callbacks=[
        DeviceStatsMonitor(cpu_stats=True),
        EarlyStopping(monitor="val_loss", patience=5), 
        checkpoint_callback
    ]
)
trainer.fit(model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type            | Params
--------------------------------------------------
0 | encoder       | Encoder         | 3.7 M 
1 | forecaster    | ForecasterPONI  | 3.9 M 
2 | discriminator | Discriminator   | 1.2 M 
3 | loss_fn       | WeightedMaeLoss | 0     
4 | dis_loss_fn   | BCELoss         | 0     
--------------------------------------------------
8.7 M     Trainable params
0         Non-trainable params
8.7 M     Total 

/wk171/handsomedong/deepQPF_research/src/utils/performance_diagram.py:192: RuntimeWarning: Mean of empty slice.
  return np.array(arr)[mask].mean()
/home/handsomedong/miniconda3/envs/deepQPF_research/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/wk171/handsomedong/deepQPF_research/src/utils/performance_diagram.py:187: RuntimeWarning: invalid value encountered in scalar divide
  output.append(2 * pd * sr / (pd + sr))


Epoch 0: 100%|██████████| 1274/1274 [10:45<00:00,  1.97it/s, v_num=16, loss_P_step=1.640, loss_GD_step=3.960, loss_G_step=1.660, loss_D_step=0.0044, loss_P_epoch=1.770, loss_GD_epoch=4.160, loss_G_epoch=1.790, loss_D_epoch=0.00702]

Epoch 0, global step 2548: 'val_loss' reached 1.76830 (best 1.76830), saving model to '/wk171/handsomedong/deepQPF_research/checkpoint/epoch=0_val_loss=1.768301.ckpt' as top 1


Epoch 1: 100%|██████████| 1274/1274 [03:56<00:00,  5.38it/s, v_num=16, loss_P_step=1.370, loss_GD_step=4.900, loss_G_step=1.410, loss_D_step=0.00251, loss_P_epoch=1.430, loss_GD_epoch=4.400, loss_G_epoch=1.460, loss_D_epoch=0.00284]

Epoch 1, global step 5096: 'val_loss' reached 1.72680 (best 1.72680), saving model to '/wk171/handsomedong/deepQPF_research/checkpoint/epoch=1_val_loss=1.726796.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 1274/1274 [03:57<00:00,  5.37it/s, v_num=16, loss_P_step=1.370, loss_GD_step=4.900, loss_G_step=1.410, loss_D_step=0.00251, loss_P_epoch=1.430, loss_GD_epoch=4.400, loss_G_epoch=1.460, loss_D_epoch=0.00284]
